# King County Home Improvements
![Hero Lake Washington, King County](images/hero-lake-washington-xlg.jpg)
<br>

**Author**: Carl Schneck <br>
**Program**: Data Science Flex <br>
**Phase 2 Project**

---

In [8]:
# Initializations
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import stats
import data_preparation_functions as dp
import figure_functions as fg

%matplotlib inline

## Overview

This project analyzes King County housing sales data in order to help lead a wholesale real estate investor make educated decisions on which home improvements best improve sales prices in the area. King County is the most populous county in Washington State, and 13th in the country.  

---

## Business Understanding

A wholesale real estate investor wants to get a better idea of which improvements relate to the the biggest increase in sales price. By finding out this information they can better access which projects are more worthwile and lead to the largest profit margin. Through a linear regression analysis we can figure out which features have the largest affect on sales price by looking at our models coefficients. 

The area of interest is King County, Washington. Which is the most populous county of Washington state, and ranked 13th in the country. In order to complete the objective a dataset containing sales data for King County spanning the years 2014 to 2015 was analyzed. 

---

## Data Understanding

Our dataset includes information on houses sold in the timeframe spanning the years 2014 through 2015 and 70 zipcodes of King County. There are a total of 21,597 entries with 21 columns worth of information. For this analysis we will cut down this data to features we believe will be helpful for the client. These include features that can be improved after the purchase of a home. Features dealing with location, view or neighbooring properties are things that in most cases are impossible to change so will be ommited.    

After taking the above into consideration we began modelling with the features below, with `price` being our target variable. For a more in depth analysis of why the other features were not included, as well as the the preperation and modeling process, please observe the EDA and Modeling notebook linked <a href="EDA_and_Modelling.ipynb">here</a>.

- `price` - Sales price of house
- `bedrooms` - Number of bedrooms
- `bathrooms` - Number of bathrooms
- `sqft_living` - Square footage of living space
- `floors` - Number of floors
- `condition` - Overall maintanence condition of the house
    - 1 = Poor
    - 2 = Fair
    - 3 = Average
    - 4 = Good
    - 5 = Very Good
- `grade` - Overall construction and design grade of the house
    - 3 = Poor
    - 4 = Low
    - 5 = Fair
    - 6 = Low Average
    - 7 = Average
    - 8 = Good
    - 9 = Better
    - 10 = Very Good
    - 11 = Excellent
    - 12 = Luxury
    - 13 = Mansion

## Data Preparation

There were a few steps taken to prepare the data. 

1. Drop the unnecessary columns
2. Convert the categorical columns to numerical columns
    - (`grade`, `condition`)
3. Drop any duplicates
4. Drop any outliers

The initial dataframe after preparation is shown below. 

In [6]:
# Loads initial prepared dataframe from the EDA_and_Modelling notebook
init_df = pd.read_pickle('data/init_df.pk1')
init_df.head(3)

,price,bedrooms,bathrooms,sqft_living,floors,condition,grade
0,221900.0,3,1.00,1180,1.0,3,7
1,538000.0,3,2.25,2570,2.0,3,7
2,180000.0,2,1.00,770,1.0,3,6


The data was then split into a training and test set with a ratio of 4:1 in order to have a dataset to validate our model.

## Modeling

In order to solve our clients problem we need to be able to infer information from our model, therefore the inner mechanisms have to be kept simple and understandable. One type of model that can fit this description is a Linear Regression Model. In theory once the model is complete we should be able to pull out the coefficients related to each feature, and have a good approximation on their relationships with the target variable. Thus this type of model was chosen for this project.

After much trial and error the final model only contained three features.

- `sqft_living`
- `grade`
- `condition`

This was mostly due to the fact that some features had high correlation to each other causing the models to perform poorly. These included the relationships of `bedrooms` and `bathrooms` with `sqft_living`. This makes sense considering they are part of the living space and adding to the square footage. The feature `floors` was later taken out due to it's high p-value, thus providing a lack of confidence in it's results.

The model also had to be limited to homes with a sales price under 1.5 million dollars. This was performed because the `price` variable had a long right tail, causing the model to lack normality of its residuals.

Scaling was also performed on the model since the variable `sqft_living` contained much higher values compared to the other two features, possibly causing a bias in the results.

The final model can be previewd below

---

In [9]:
# Loads final model training sets
X_train = pd.read_pickle('data/X_train_final.pk1')
y_train = pd.read_pickle('data/y_train_final.pk1')

In [13]:
# Creates and previews final model
model = dp.model_summary(X_train, y_train)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.521
Model:                            OLS   Adj. R-squared:                  0.521
Method:                 Least Squares   F-statistic:                     6068.
Date:                Tue, 28 Mar 2023   Prob (F-statistic):               0.00
Time:                        02:09:52   Log-Likelihood:            -2.2574e+05
No. Observations:               16760   AIC:                         4.515e+05
Df Residuals:                   16756   BIC:                         4.515e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -1.387e+05   7379.186    -18.795      0.000   -1.53e+05   -1.24e+05
sqft_living  7.439e+05   1.68e+04     44.408      0.000    7.11e+05    7.77e+05
condition     1.97e+05   8317.684     23.685      0.000    1.81e+05    2.13e+05
grade         7.75e+05   1.43e+04     54.120      0.000    7.47e+05    8.03e+05
==============================================================================
Omnibus:                     2333.887   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4494.887
Skew:                           0.881   Prob(JB):                         0.00
Kurtosis:                       4.826   Cond. No.                         20.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [17]:
model.params.sort_values(ascending=False)

grade          774981.565432
sqft_living    743945.080512
condition      197005.386528
const         -138691.105288
dtype: float64

The top two features from this model are `sqft_living` and `grade`, thus will be the recommended features to improve to reach the greatest profit. 

Since the model was scaled these coefficients cannot be directly used. The independant features of the model were scaled using the formula below.

<br>
<center>$$x_{iscaled} = \displaystyle \frac{x_{i} - x_{min}}{x_{max} - x_{min}}$$</center>
<br>

- $x_{iscaled}$ is the scaled value of $x_{i}$
- $x_{i}$ are the individual values in feature x
- $x_{min}$ is the minimun value in feature x
- $x_{max}$ is the maximum value in feature x

What's important for our analysis is the rate of change. Since the transformation was linear we only need to take the denominator into consideration. Thus the rate of change per unit of the feature can be found by using the equation below, where c is the coeffiecient of the feature calculated by the model.

<br>
<center>$$rate = \displaystyle \frac{c}{x_{max} - x_{min}}$$</center>
<br>

The rate of change per unit has been saved in the dataseries below.

In [24]:
# Loads rates of change per unit from pickle file
rates = pd.read_pickle('data/rates.pk1')

## Evaluation

---

## Conclusion

---

## Appendix
Extra Figures??

---

## Resources

Any Works Cited

---